In [1]:
import sys
import os
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores.chroma import Chroma
from langchain_chroma import Chroma


_=load_dotenv(find_dotenv())

# 定义 Embeddings
embedding = OpenAIEmbeddings()

deepseek_api_key=os.environ['DEEPSEEK_API_KEY']

In [3]:
# 向量数据库持久化路径
persist_directory = 'vector_db/chroma'

In [4]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [5]:
#向量库中存储的数量
vectordb._collection.count()

721

In [7]:
vectordb.similarity_search("llm", k=3)

[Document(id='37a8529a-c1e5-4b90-8be4-7884b2986399', metadata={'source': 'files\\必修一-Prompt Engineering For Developers\\1. 简介.md'}, page_content='随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前,有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如果你以“法国的首都是什么”为 Prompt ，则基础 LLM 可能会根据互联网上的文章，将回答预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。'),
 Document(id='2bbd288a-52dc-4eaf-a629-a30ad5aceefc', metadata={'source': 'files\\必修一-Prompt Engineering For Developers\\1. 简介.md'}, page_content='而对于指令微调的 LLM ，相关研究和实践正甚嚣尘上，训练它们来遵循指示。因此，如果你问它，“法国的首都是什么？”，它有极大可能输出“法国的首都是巴黎”。指令微调的LLM的训练通常是基于预训练好的LLM的，即模型已经在大量文本数据上进行了训练。然后对其进行进一步训练与微调（finetune），使用的数据包括输入和理想输出（输入是指令、输出是遵循这些指令的良好回答）。然后通常使用一种称为 RLHF（reinforcement learning from human feedback，人类反馈强化学习）的技术进行进一步改进，使系统更能够有帮助地遵循指令。\n\n因为指令微调的 LLM 已经被训练成有益、诚实、无害的，所以与基础 LLM 相比，它们更不可能输出有问题的文本，如有害输出。许多实际使用场景已经转向指令微调的 LLM 。您在互联网上找到的一些最佳实践可能更适用于基础 LLM ，但对于今天的大多

In [8]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")


In [17]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "你是谁?"},
    ],
    stream=False,
    temperature=1
)

print(response.choices[0].message.content)

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-V3。如您有任何任何问题，我会尽我所能为您提供帮助。


In [12]:
from langchain.prompts import PromptTemplate

In [27]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [28]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("I fucked your mom last night！")

AIMessage(content="I'm sorry, but that kind of language is not appropriate or respectful. Let's keep the conversation civil and respectful.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 14, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e5a6ddc-876c-4ac1-b9ef-d9477f235cf3-0', usage_metadata={'input_tokens': 14, 'output_tokens': 25, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [30]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

In [32]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [34]:
result = qa_chain({"query": "什么是南瓜书？"})
result

{'query': '什么是南瓜书？',
 'result': '南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的书籍。\n谢谢你的提问！',
 'source_documents': [Document(id='fab1fd3c-ecf2-42dc-b4fe-5ba7b01f2d00', metadata={'author': '', 'creationDate': "D:20231117152045-00'00'", 'creationdate': '2023-11-17T15:20:45+00:00', 'creator': 'LaTeX with hyperref', 'file_path': 'files\\pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'moddate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': 'files\\pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜

In [35]:
result = qa_chain({"query": "Prompt Engineering for Developer是谁写的？"})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
Prompt Engineering for Developer是由吴恩达教授和Isa Fulford教授共同编写的。谢谢你的提问！


In [37]:
prompt_template = """请回答下列问题:
                            {}""".format("Prompt Engineering for Developer是谁写的？")

### 基于大模型的问答
llm.predict(prompt_template)

C:\Users\AriZu\AppData\Local\Temp\ipykernel_18672\924274261.py:5: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict(prompt_template)


'Prompt Engineering for Developer是由一位名叫Yufan Lu的作者撰写的。'